In [7]:
import json
import numpy as np
import tensorflow as tf
import keras
from keras.utils import image_dataset_from_directory
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# WITH PREPROCESSING

In [8]:
ds_train = image_dataset_from_directory('../data/french_bird_db_200_400', labels='inferred', image_size=(300, 300), validation_split=0.2, subset="training",
seed=42, batch_size=320)

ds_test = image_dataset_from_directory('../data/french_bird_db_200_400', labels='inferred', image_size=(300, 300), validation_split=0.2, subset="validation",
seed=42, batch_size=320)

Found 80000 files belonging to 200 classes.
Using 64000 files for training.
Found 80000 files belonging to 200 classes.
Using 16000 files for validation.


In [9]:
class_names = ds_train.class_names
num_classes = len(ds_train.class_names)
with open('class_names.json', 'w') as f:
    json.dump(class_names, f)

In [10]:
# preprocessing
preprocess_input = tf.keras.applications.resnet50.preprocess_input

ds_train_preprocessed = ds_train.map(lambda x, y: (preprocess_input(x), y))
ds_test_preprocessed = ds_test.map(lambda x, y: (preprocess_input(x), y))


In [11]:
tf.config.list_physical_devices()

# Charger le modèle ResNet50 pré-entraîné avec les poids ImageNet
base_model = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=(300, 300, 3))

# Ajouter une couche de pooling global pour réduire la dimensionalité
x = base_model.output
x_2 = GlobalAveragePooling2D()(x)

x_3 = Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))(x_2)
x_3 = Dropout(0.5)(x_3)


# Ajouter une couche dense pour effectuer la classification
output = Dense(num_classes, activation='softmax')(x_3)

# Créer le modèle final en combinant le modèle de base et les couches supplémentaires
model = Model(inputs=base_model.input, outputs=output)

# Geler les couches du modèle de base pour éviter de les entraîner
for layer in base_model.layers:
    layer.trainable = False

# Compiler le modèle
model.compile(optimizer=Adam(learning_rate=0.001), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=[keras.metrics.SparseCategoricalAccuracy()])

callbacks = [EarlyStopping(monitor='val_loss', patience=10)]


# Entraîner le modèle sur votre dataset avec le callback personnalisé
history = model.fit(ds_train_preprocessed, epochs=5, batch_size=320, callbacks=callbacks, validation_data=ds_test_preprocessed)

Epoch 1/5


2024-07-04 16:05:47.897191: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


  9/200 ━━━━━━━━━━━━━━━━━━━━ 28:11 9s/step - loss: 13.1793 - sparse_categorical_accuracy: 0.0081

KeyboardInterrupt: 

In [6]:
model.save_weights('current_model.weights.h5')

NameError: name 'model' is not defined

In [ ]:
image_test = keras.utils.load_img(
    '../data/french_bird_db_50_100/apus_apus/apus_apus_10.jpg',
)

image_array = keras.utils.img_to_array(image_test)

image_processed = preprocess_input(image_array)
# ds_train_preprocessed = ds_train.map(lambda x, y: (preprocess_input(x), y))
print(image_processed.shape)

test = np.argmax(model.predict(np.expand_dims(image_processed, axis=0)))

print(test, class_names[test])

(300, 300, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
42 spatula_querquedula


In [ ]:
for image_batch, labels_batch in ds_train_preprocessed:
  batch_pred = model.predict(image_batch)
  class_pred = np.argmax(batch_pred, axis=1)
  print(class_pred)
  print(labels_batch)
  break

2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 176ms/step
[29 31  6 38 23 35 10 36  0 20 35 17  7 22 32 36  4  5  4 46 29 39 37  1
  1  3 37 37 47 45 39 21 39 32 11  4 20 41 28 15 18 13 36 42  4 30 15 42
 14 41 49 28 18 30 31 46 29 40  1 26 26 42 26 34]
tf.Tensor(
[29 31  6 38 23 35 10 28  0  6 35 17  7 22 32 16  4  5  4 46 23 39 37 27
 28  3 37 37 47 16 39 21 39 32 11  4 20 41 28 15 18 13 36 43  4 30 15 38
 14 41 49 27 18 30 31 46 30 40  1 26 26 49 26 34], shape=(64,), dtype=int32)


In [ ]:
total = 0
for i, (image_batch, labels_batch) in enumerate(ds_test_preprocessed):
  batch_pred = model.predict(image_batch)
  max = np.max(batch_pred, axis=1)
  
  print('batch No {} with result: '.format(i), max * 100, '%')
  


2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 722ms/step
batch No 0 with result:  [92.33191  50.998886 94.86049  99.334915 47.39811  74.51995  85.06272
 62.211735 30.249184 95.44793  27.761698 22.99988  88.10974  51.252705
 92.94134  88.10425  42.539272 39.783073 85.60944  65.02477  31.592268
 30.340868 36.202084 28.147339 81.10364  93.99651  73.191795 72.8503
 89.825005 86.84129  62.19917  59.047256 97.512924 39.510487 88.72931
 87.14251  51.50335  77.28859  55.37567  51.019787 44.48908  99.6562
 82.17718  50.671024 90.80182  85.999466 43.20118  30.971241 98.78494
 92.87075  78.97164  88.769615 82.06567  29.477167 40.842495 87.92472
 76.71177  86.73783  99.663475 97.754524 91.146675 99.167564 51.230217
 89.55529 ] %
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 683ms/step
batch No 1 with result:  [89.73922  41.327324 40.21643  79.10757  42.733063 91.43583  96.009125
 22.446022 74.80963  79.72755  52.804817 33.536697 92.71812  71.357185
 23.531012 68.89083  94.845024 41.654507 49.17503  98.281136 90.06487
 55.208023 22.3

2024-07-04 15:28:24.533678: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [ ]:
total = 0
for i, (image_batch, labels_batch) in enumerate(ds_test_preprocessed):
  batch_pred = model.predict(image_batch)
  class_pred = np.argmax(batch_pred, axis=1)
  res = sum((class_pred == labels_batch).numpy()) / len(class_pred)
  total += res
  
  print('batch No {} with result: '.format(i), res * 100, '%')
  
print('FINAL RES = ', total / len(ds_test_preprocessed))

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 633ms/step
batch No 0 with result:  84.375 %
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 644ms/step
batch No 1 with result:  78.125 %
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 646ms/step
batch No 2 with result:  70.3125 %
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 662ms/step
batch No 3 with result:  70.3125 %
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 653ms/step
batch No 4 with result:  81.25 %
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 655ms/step
batch No 5 with result:  79.6875 %
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 650ms/step
batch No 6 with result:  76.5625 %
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 682ms/step
batch No 7 with result:  75.0 %
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 666ms/step
batch No 8 with result:  73.4375 %
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 655ms/step
batch No 9 with result:  70.3125 %
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 681ms/step
batch No 10 with result:  78.125 %
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 664ms/step
batch No 11 with result:  79.6875 %
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 660ms/step
batch No 12 with result:  67.1875 %
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 663ms/step
batc

2024-07-04 14:58:08.931451: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
